In [1]:
import pandas as pd
from collections import Counter
import re
import pymorphy2
from nltk import word_tokenize
from nltk.util import ngrams
from nltk.corpus import stopwords


# Очистка и обработка данных

## Очистка

In [27]:
path = "your/path/to/data.csv"
df = pd.read_csv(path)

In [28]:
# выделяем из базы данных только те колонки, которые нам нужны
# выбираем сообщения, опубликованные не раньше 1 января 2024 года
df = df[['date', 'message_id', 'message']]
df = df.loc[df['date'] >= '2024-11-19']

In [35]:
# удаляем ненужные строки, рекламу
ad_phrases = [
                'mediamessage',
                'дайджест',
                '#дайджест',
                'платноеразмещение',
                'партнерский',
                'рекламная_подборка',
                '#реклама',
                '#портретрегиона',
                'о своих проектах на нашу аудиторию',
                'произошло в регионах',
                'новости в редакцию «7х7»',
                'подари «7х7» звезды',
                'звонок для учителя',
                'это анонимный автор',
                'ежемесячное пожертвование',
                'зарубежной карты',
                'время прочтения',
                'для чтения в telegram',
                'время чтения',
                'к нам пришла женщина',
                '#естьвопрос',
                '#музыка',
                '#книги',
                'легенды7х7',
                '#текст_немосква',
                '#Герои7',
                'Наш эфир',
                'эфир «Эха регионов»',
                'Читайте текст в «Новой вкладке»'
]

# функция для классификации рекламных сообщений
# на входе: текст сообщения, на выходе: маркировка сообщения
def is_ad(text):
    if not isinstance(text, str):  # Handle non-string values
        return False
    return any(phrase in text.lower() for phrase in ad_phrases)

# удаляем все сообщения, маркированные как реклама
df = df[~df['message'].apply(is_ad)]

## Обработка

In [36]:
# приведем все буквы в нижний регистр
df['message'] = df['message'].str.lower()

In [37]:
# выделим заголовки из сообщений
# на входе: сообщение, где предложения выделены ** и остальной текст
# на выходе: только предложения между **
def extract_title(message):
    if not isinstance(message, str):  # Handle non-string values
        return None
    match = re.search(r'\*\*(.*?)\*\*', message)
    return match.group(1).strip() if match else None

df['title'] = df['message'].apply(extract_title)

In [39]:
# достаем из сообщений все хэштеги
def extract_hashtags(text):
    if not isinstance(text, str):  # если попадется не строка
        return []
    return re.findall(r'#\w+', text)  # выделаем все слова, перед которыми стоит #

df['hashtags'] = df['message'].apply(extract_hashtags)

In [40]:
# после того, как достали заголовки и хэштеги, удалим ненужные символы
df['message'] = df['message'].str.replace(r'[^а-яА-ЯёЁ\s]', ' ')

/var/folders/5r/7bhf5tqj0wb9bbk62kmr6wk80000gp/T/ipykernel_15882/2199477774.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['message'] = df['message'].str.replace(r'[^а-яА-ЯёЁ\s]', ' ')


In [41]:
# приводим все слова в сообщениях в инфинитивную форму, 
# чтобы легче было использовать фильтр 
morph = pymorphy2.MorphAnalyzer()

def normalize_text(text):
    if not isinstance(text, str):  # если попадется не строка
        return ""
    tokens = text.split()  # разделяем слова по пробелам
    normalized_tokens = [morph.parse(word)[0].normal_form for word in tokens]  # приводим в инфинитив
    return " ".join(normalized_tokens)  # соединяем обратно инфинитивы в сообщение

In [42]:
df['normalized_message'] = df['message'].apply(normalize_text)

In [43]:
# приводим слова в заголовках в инфинитивную форму
df['title'] = df['title'].str.replace(r'[^а-яА-ЯёЁ\s]', ' ')
df['title'] = df['title'].apply(normalize_text)

/var/folders/5r/7bhf5tqj0wb9bbk62kmr6wk80000gp/T/ipykernel_15882/2464783884.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['title'] = df['title'].str.replace(r'[^а-яА-ЯёЁ\s]', ' ')


## Дополнительная очистка данных

In [47]:
df = df[~df['normalized_message'].str.contains('городской легенда')]
df = df[~df['normalized_message'].str.contains('выбирать победитель премия')]
df = df[~df['normalized_message'].str.contains('эфир эхо регион')]
df = df[~df['normalized_message'].str.contains('путевой дневник')]
df = df[~df['normalized_message'].str.contains('наш уличный опрос')]
df = df[~df['message'].str.contains(r'подводим итоги\s+\d+\s+голосования')]

In [92]:
# оставляем в колонке с хэштегами только первый хэштег
# очищаем получившееся от лишних знаков
df['hashtags'] = df['hashtags'].str.split(',').str[0]
df['hashtags'] = df['hashtags'].str.replace('#', '')

# Категоризация данных

## Категоризуем сообщения

In [94]:
category_keywords = {

'legal': [
    'штраф',
    'дискредитация',
    'административный протокол',
    'административный дело',
    'уголовный дело',
    'лишение свобода',
    'предостережение',
    'овд инфо',
    'стать свидетель',
    'cуд',
    'приговорить год колония',
    'оштрафовать',
    'колония'
],
'protest' : [
    'акция',
    'протест',
    'пикет',
    'митинг',
    'против',
    'забастовка',
    'петиция',
    'сбор',
    'подпись',
    'письмо',
    'отставка',
    'бунт',
    'обращение',
    'голодовка',
    'набрать',
    'стихийный мемориал',
    'нести цветок',
    'сход',
    'сбор подпись',
    'в знак протест',
    'тысяч подпись',
    r'\bмитинг против отмена',
    'запустить петиция'
    # 'правозащитник'
    # 'против строительство',
    # 'выступить против',
    # 'противн',
    # r'[\s]увол[^\s]+',
    # 'антивоен',
    # r'активист',
    # r'[\s]задерж[^\s]+',
    # r'штраф',
    # r'[\s]уйти в отставку[^\s]+',
    # r'[\s]сбор подписей[^\s]+',
    # r'[\s]к Путину[^\s]+',
    # r'[\s]потребовали отставки[^\s]+',
],
'elections' : [
    'избирательный участок',
    'борис надеждин',
    'штаб надеждин',
    r'надеждин[^\s]+',
    'выборы',
    'полдень против путин'
],
'terrorism' : [
    'крокус сити',
    'сити холл',
    'теракт крокус',
    'крокус сити холл'
]

}

In [64]:
hashtag_keywords = [
    '#выборы2024',
    '#выборы24',
    '#__выборы24',
    '#выборы____',
    '#выборы24__'
]

In [65]:
def is_election(text):
    if text is None:  # если вдруг нет текста
        return None
    return 'elections' if any(keyword in text for keyword in hashtag_keywords) else None

df['category'] = df['hashtags'].apply(is_election)

In [66]:
# категоризуем сообщения, если у них ещё нет категории
def categorize_message(text, current_category):
    if current_category and current_category != "Uncategorized":  # пропускаем уже категаризованные сообщения
        return current_category
    if not isinstance(text, str):  # а вдруг не строка
        return "Uncategorized"
    for category, keywords in category_keywords.items():
        if any(keyword in text for keyword in keywords):  # сопоставляем со словарём
            return category
    return "Uncategorized"  


In [67]:
df['category'] = df.apply(lambda row: categorize_message(row['normalized_message'], row['category']), axis=1)

## Категоризуем заголовки

In [68]:
# используем тот же словарь для категоризации заголовков
def categorize_title(text):
    if not isinstance(text, str):  
        return "Uncategorized"
    for category, keywords in category_keywords.items():
        if any(keyword in text for keyword in keywords):  
            return category
    return "Uncategorized"

In [69]:
df['category_title'] = df['title'].apply(categorize_title)

# Более точный фильтр

In [100]:
# функция для расчёта "протестности" сообщения
def protest_filter(row, threshold=4.0):
    message = row['message'] if pd.notna(row['message']) else ""
    title = row['title'] if pd.notna(row['title']) else ""
    hashtags = row['hashtags'] if pd.notna(row['hashtags']) else ""

    # всё ещё раз в нижний регистр
    message, title, hashtags = message.lower(), title.lower(), hashtags.lower()

    # фразы о протестах
    high_weight_phrases = {
        r"потребовали возбудить уголовное дело": 4.5,
        r"митинг прошел": 4.5,
        r"создали петицию против": 4.5,
        r"\bчеловек (вышли|собрались) на митинг против\b": 4.5,
        r"\bжителей .+? потребовали отставки\b": 4.5,
        r"\bмитинг против\b": 4.0,
        r"\bначали сбор подписей\b": 4.5,
        r"собрали тыс подписей": 4.5,
        r"вышли на митинг(и)? против": 4.5,
        r"вышли стихийный митинг": 4.5,
        r"вышли пикет против": 4.5,
        r"\bпотребовало отставки\b": 4.0,
        r"потребовали уволить": 4.0,
        r"запустили петицию против": 4.0,
        r"вышли сход против": 4.0,
        r"протестуют против": 4.0,
        r"провел одиночный пикет": 3.5,
        r"\bустроил одиночный пикет\b": 3.5,
        r"горожане беспокоятся": 3.5,
        r"сбор подписей отставку": 4.0,
        r"жители выступили против": 4.0,
        r"привлечь внимание проблеме": 3.5,
        r"собирать подписи": 4.0,
        r"протест против": 4.0,
        r"тысяч(и)? подписей": 4.0,
        r"вышли пикет": 4.0,
        r"вышли сход": 4.0,
        r"записали видеообращение": 4.0,
        r"акцию протеста": 4.0,
        r"петицию c требованием": 3.5,
        r"сбор подписей": 4.0,
        r"одиночный пикет": 4.0
    }

    # фразы не о протестах
    non_protest_phrases = {
        r"вопреки протестам жителей": -5.0,
        r"подал(а|и)? заявку на митинг": -5.0,
        r"хотят устроить акцию": -5.0,
        r"не (согласовали|разрешили) митинг": -5.0,
        r"(согласовал(и|а)|разрешил(и|а)?) митинг": -5.0,
        r"выйдут на митинг": -5.0,
        r"запланировал(и)? митинг": -5.0,
        r"заставили\w+подписать": -5.0,
        r"(поджог|поджег|подожгл(а|и))\w+военкомат(а)?": -5.0,
        r"после недовольства жителей": -5.0,
        r"убрали стихийный мемориал": -5.0,
        r"антивоенны\s+ пост(ы)?": -5.0,
        r"оштрафовали за пост(ы)?": -5.0,
        r"несмотря на\s+\d+\s+тысяч подписей": -5.0,
        r"надеждин собрал больше\s+\d+\s+тысяч подписей": -5.0,
        r"власти\s+\d+\s+остановили\s+\d+\s+по просьбе жителей": -5.0,
        r"из за забастовки": -5.0,
        r"подводим итоги": -5.0,
        r"давно жаловались": -5.0,
        r"из за требований": -5.0,
        r"вопреки мнению жителей": -5.0,
        r"пост в канале": -5.0,
        r"после видеообращения (к)? путину": -5.0,
        r"власти опасаются акций протеста": -5.0,
        r"раскритиковали": -3.0
    }

    # ключевые слова о протестах
    protest_keywords = {
        r"\bмитинг\b": 2.5,
        r"\bпикет\b": 2.5,
        r"\bпетиция\b": 3.0,
        r"\bзабастовка\b": 3.0,
        r"\bпротест[а-я]*\b": 3.5,
        r"\bжители\b": 2.5,
        r"\bвозложить цветы\b": 3.0,
        r"\bгубернатор\b": 1.5,
        r"\bвласть\b": 1.5,
        r"\bстроительство\b": 1.0,
        r"\bпротив\b": 1.0,
        r"\bзакрытие\b": 1.5
    }

    election_hashtags = [r"#выборы(?:2024|24|__.*)?"]
    terrorism_keywords = [r"крокус сити", r"теракт", r"сити холл"]

    score = 0.0

    # проверяем на наличие протестных фраз
    for pattern, weight in high_weight_phrases.items():
        if re.search(pattern, message) or re.search(pattern, title):
            score += weight

    # проверяем на наличие фраз не о протестах
    for pattern, weight in non_protest_phrases.items():
        if re.search(pattern, message) or re.search(pattern, title):
            score += weight

    # проверяем по протестных словам
    for pattern, weight in protest_keywords.items():
        if re.search(pattern, message) or re.search(pattern, title):
            score += weight

    # понижаем значимость новостей о выборах (обычно не про протесты)
    for pattern in election_hashtags:
        if re.search(pattern, hashtags):
            score -= 3.5 

    # понижаем значимость новостей о теракте (обычно не про протесты)
    for pattern in terrorism_keywords:
        if re.search(pattern, message) or re.search(pattern, title):
            score -= 3.5  

    # подводим итог
    return "yes" if score >= threshold else "no"

In [101]:
# применяем фильтр
df['threshold 4.0'] = df.apply(protest_filter, axis=1)

# Проверка фильтра

In [7]:
labeled_data = df[df['theme by hand'].notna()]
correct_predictions = (labeled_data['Refined_Filter'] == labeled_data['theme by hand']).sum()
total_predictions = len(labeled_data)
accuracy = correct_predictions / total_predictions

accuracy

0.6047819971870605

# География протестов

In [116]:
# по хэштегам определяем регион
hashtag_to_region = {
    "адыгея": "Adygea",
    "алтай": "Altai",
    "алтайскийкрай": "Altai Krai",
    "амурскаяобласть": "Amur Region",
    "архангельскаяобласть": "Arkhangelsk Region",
    "арханельскаяобласть": "Arkhangelsk Region",
    "астраханскаяобласть": "Astrakhan Region",
    "башкортостан": "Bashkortostan",
    "белгородскаяобласть": "Belgorod Region",
    "брянскаяобласть": "Bryansk Region",
    "бурятия": "Buryatia",
    "владимирскаяобласть": "Vladimir Region",
    "волгоградскаяобласть": "Volgograd Region",
    "вологодскаяобласть": "Vologda Region",
    "воронежкаяобласть": "Voronezh Region",
    "воронежскаяобласть": "Voronezh Region",
    "дагестан": "Dagestan",
    "еао": "Jewish Autonomous Region",
    "забайкалье": "Zabaykalsky Krai",
    "забайкальскийкрай": "Zabaykalsky Krai",
    "ивановскаяобласть": "Ivanovo Region",
    "ингушетия": "Ingushetia",
    "иркутскаяобласть": "Irkutsk Region",
    "ирткутскаяобласть": "Irkutsk Region",
    "кабардинобалкария": "Kabardino-Balkaria",
    "калининградскаяобласть": "Kaliningrad Region",
    "калмыкия": "Kalmykia",
    "калужскаяобласть": "Kaluga Region",
    "камчатка": "Kamchatka Krai",
    "камчатскийкрай": "Kamchatka Krai",
    "карачаевочеркесия": "Karachay-Cherkessia",
    "карелия": "Karelia",
    "кбр": "Kabardino-Balkaria",
    "кемеровскаяобласть": "Kemerovo Region",
    "кировскаяобласть": "Kirov Region",
    "коми": "Komi",
    "костромскаяобласть": "Kostroma Region",
    "краснодарскийкрай": "Krasnodar Krai",
    "красноярскийкрай": "Krasnoyarsk Krai",
    "курганскаяобласть": "Kurgan Region",
    "курскаяобласть": "Kursk Region",
    "ленинградскаяобласть": "Leningrad Region",
    "липецкаяобласть": "Lipetsk Region",
    "магаданскаяобласть": "Magadan Region",
    "марийэл": "Mari El",
    "мордовия": "Mordovia",
    "москва": "Moscow",
    "московскаяобласть": "Moscow Region",
    "мурманскаяобласть": "Murmansk Region",
    "нао": "Nenets Autonomous Okrug",
    "нижегородскаяобласть": "Nizhny Novgorod Region",
    "новгородскаяобласть": "Novgorod Region",
    "новосибирскаяобласть": "Novosibirsk Region",
    "омскаяобласть": "Omsk Region",
    "оренбургскаяобласть": "Orenburg Region",
    "орловскаяобласть": "Oryol Region",
    "орловскаябласть": "Oryol Region",
    "пензенскаяобласть": "Penza Region",
    "пермскийкрай": "Perm Krai",
    "приморскийкрай": "Primorsky Krai",
    "псковскаяобласть": "Pskov Region",
    "республикаалтай": "Republic of Altai",
    "ростовскаяобласть": "Rostov Region",
    "рязанскаяобласть": "Ryazan Region",
    "самарскаяобласть": "Samara Region",
    "санктпетербург": "Saint Petersburg",
    "саратовскаяобласть": "Saratov Region",
    "сахалин": "Sakhalin Region",
    "сахалинскаяобласть": "Sakhalin Region",
    "свердловскаяобласть": "Sverdlovsk Region",
    "севернаяосетия": "North Ossetia",
    "смоленскаяобласть": "Smolensk Region",
    "ставрапольскийкрай": "Stavropol Krai",
    "ставропольскийкрай": "Stavropol Krai",
    "тамбовскаяобласть": "Tambov Region",
    "татарстан": "Tatarstan",
    "тверскаяобласть": "Tver Region",
    "томскаяобласть": "Tomsk Region",
    "тульскаяобласть": "Tula Region",
    "тыва": "Tuva",
    "тюменскаяобласть": "Tyumen Region",
    "удмуртия": "Udmurtia",
    "ульяновскаяобласть": "Ulyanovsk Region",
    "хабаровскийкрай": "Khabarovsk Krai",
    "хакасия": "Khakassia",
    "хмао": "Khanty-Mansi Autonomous Okrug",
    "челябинскаяобласть": "Chelyabinsk Region",
    "чечня": "Chechnya",
    "чувашия": "Chuvashia",
    "чукотка": "Chukotka",
    "якутия": "Sakha (Yakutia)",
    "янао": "Yamalo-Nenets Autonomous Okrug",
    "ярославскаяобласть": "Yaroslavl Region"
}

# функция для определения региона
def map_to_region(hashtags):
    if pd.isna(hashtags):
        return "Unknown"
    for hashtag in hashtags.split(","):
        hashtag = hashtag.strip().lower()
        if hashtag in hashtag_to_region:
            return hashtag_to_region[hashtag]
    return "Other"

In [117]:
df['geographic_region'] = df['hashtags'].apply(map_to_region)